In [1]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-10-18 10:19:25--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-10-18 10:19:25 (18.6 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [2]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [3]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [8]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [5]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [11]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [12]:
llm("write this is as a test")

"Test Instruction:\nDevelop and analyze an algorithm that generates all possible permutations of the input string, excluding any repetitions. Provide time complexity and space usage analysis for your solution in Big-Oh notation both iteratively using recursive backtracking techniques with memoization as well as via a non-recursive method employing Heap's Algorithm to optimize recursion overhead while avoiding stack overflow or excessive temporary storage allocation beyond system capacities. Ensure that no duplicate permutations are generated, even if the input string contains characters multiple times for instance 'aabb'. After establishing both recursive and iterative approaches with their efficiency analysis based on your established notions of time complexity (T) in Big-Oh notation as O(n!) where n is the length of the longest possible sequence without duplication. Space Complexity should reflect maximum memory consumption during worst case scenarios, represented using Sigma notatio

In [13]:
print(_)

Test Instruction:
Develop and analyze an algorithm that generates all possible permutations of the input string, excluding any repetitions. Provide time complexity and space usage analysis for your solution in Big-Oh notation both iteratively using recursive backtracking techniques with memoization as well as via a non-recursive method employing Heap's Algorithm to optimize recursion overhead while avoiding stack overflow or excessive temporary storage allocation beyond system capacities. Ensure that no duplicate permutations are generated, even if the input string contains characters multiple times for instance 'aabb'. After establishing both recursive and iterative approaches with their efficiency analysis based on your established notions of time complexity (T) in Big-Oh notation as O(n!) where n is the length of the longest possible sequence without duplication. Space Complexity should reflect maximum memory consumption during worst case scenarios, represented using Sigma notation 